# Task 4.10 - Part 1.2 - Optimizing

## 1.- Importing Libraries & Dataframe 

In [4]:
#importing libraries 
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [12]:
# Importing dataframe 

path = r'C:\Users\cmfag\OneDrive\Desktop\Data Analyst Specialist\Achievement 4\02 DATA\Prepared Data'

df_combined_filtered = pd.read_pickle(os.path.join(path,'Data_combined_15_11.pkl'))

# 1.- Optimizing the Data Size

Since i´m struggeling with the RAM memory and the high weight of the Data analized, i need to take some actions in order to use the data.

## 1.1- Action1 - Removing some Columns

In [8]:
print(df_combined_filtered.columns)

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'Unnamed: 0', 'product_name',
       'aisle_id', 'department_id', 'prices', 'merge_check', 'day_label',
       'busiest_day', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'average_spend', 'spender_flag', 'Customer_frequency', 'frequency_flag',
       'gender', 'state', 'age', 'date_joined', 'num_dependents',
       'family_status', 'income', 'Region', 'exclusion_flag'],
      dtype='object')


In [10]:
# Total size of DataFrame
df_size = df_combined_filtered.memory_usage(deep=True).sum() / 1024**2
print(f"Total Size DataFrame: {df_size:.2f} MB")

# RAM Memory usage
column_sizes = df_combined_filtered.memory_usage(deep=True).sort_values(ascending=False) / 1024**2
print("Memory usaged (in MB):")
print(column_sizes)


Total Size DataFrame: 30184.38 MB
Memory usaged (in MB):
product_name              2189.641903
exclusion_flag            2187.269388
frequency_flag            1955.305842
loyalty_flag              1883.274947
busiest_period_of_day     1832.577055
busiest_day               1831.373729
day_label                 1831.373729
spender_flag              1774.037522
date_joined               1712.381796
family_status             1708.444801
state                     1704.596956
Region                    1623.277925
eval_set                  1596.115499
gender                    1595.916446
product_id                 236.461555
income                     236.461555
num_dependents             236.461555
age                        236.461555
user_id                    236.461555
Customer_frequency         236.461555
average_spend              236.461555
days_since_prior_order     236.461555
max_order                  236.461555
order_number               236.461555
order_dow                  236.

In [15]:
# Dropping some columns

columns_to_drop = [
    'product_name', 
    'day_label',
    'date_joined',
    'eval_set',
    'product_id',
    'user_id',
    'days_since_prior_order', 
    'order_id', 
    'aisle_id',
    'Unnamed: 0',
    'reordered',
    'add_to_cart_order',
    'merge_check'
    ]

# Crear un nuevo DataFrame sin estas columnas
df_combined_filtered_reduced = df_combined_filtered.drop(columns=columns_to_drop)

# Verificar las columnas restantes
print("Columnas restantes en el DataFrame:")
print(df_combined_filtered_reduced.columns)

Columnas restantes en el DataFrame:
Index(['order_number', 'order_dow', 'order_hour_of_day', 'department_id',
       'prices', 'busiest_day', 'busiest_period_of_day', 'max_order',
       'loyalty_flag', 'average_spend', 'spender_flag', 'Customer_frequency',
       'frequency_flag', 'gender', 'state', 'age', 'num_dependents',
       'family_status', 'income', 'Region', 'exclusion_flag'],
      dtype='object')


In [17]:
# Total size of DataFrame
df_size = df_combined_filtered_reduced.memory_usage(deep=True).sum() / 1024**2
print(f"Total Size DataFrame: {df_size:.2f} MB")

# RAM Memory usage
column_sizes = df_combined_filtered_reduced.memory_usage(deep=True).sort_values(ascending=False) / 1024**2
print("Memory usaged (in MB):")
print(column_sizes)


Total Size DataFrame: 20933.61 MB
Memory usaged (in MB):
exclusion_flag           2187.269388
frequency_flag           1955.305842
loyalty_flag             1883.274947
busiest_period_of_day    1832.577055
busiest_day              1831.373729
spender_flag             1774.037522
family_status            1708.444801
state                    1704.596956
Region                   1623.277925
gender                   1595.916446
prices                    236.461555
department_id             236.461555
max_order                 236.461555
order_hour_of_day         236.461555
average_spend             236.461555
order_number              236.461555
Customer_frequency        236.461555
order_dow                 236.461555
age                       236.461555
num_dependents            236.461555
income                    236.461555
Index                     236.461555
dtype: float64


### Result Action 1: Reduction of almost 1000 MB. But since we will create some new columns in the next points, a even higher reduction will be needed. 

## 1.2.Action 2- Optimizing the data type of some columns

In [31]:
# Verifiying the data types of each column

print(df_combined_filtered_reduced.dtypes)


order_number               int64
order_dow                  int64
order_hour_of_day          int64
department_id            float64
prices                   float64
busiest_day               object
busiest_period_of_day     object
max_order                  int64
loyalty_flag              object
average_spend            float64
spender_flag              object
Customer_frequency       float64
frequency_flag            object
gender                    object
state                     object
age                        int64
num_dependents             int64
family_status             object
income                     int64
Region                    object
exclusion_flag            object
dtype: object


In [40]:
# Optimize data type
def optimize_numeric_columns_precise(df):
    initial_size = df.memory_usage(deep=True).sum() / 1024**2
    print(f"Initial Size of the frame: {initial_size:.2f} MB")
    
    for col in df.select_dtypes(include=['int64', 'float64']).columns:
        c_min = df[col].min()
        c_max = df[col].max()

        # Optimizar columnas flotantes
        if pd.api.types.is_float_dtype(df[col]):
            df[col] = df[col].astype('float32')

        # Optimizar columnas enteras
        elif pd.api.types.is_integer_dtype(df[col]):
            if c_min > -128 and c_max < 127:
                df[col] = df[col].astype('int8')
            elif c_min > -32768 and c_max < 32767:
                df[col] = df[col].astype('int16')
            elif c_min > -2147483648 and c_max < 2147483647:
                df[col] = df[col].astype('int32')

    final_size = df.memory_usage(deep=True).sum() / 1024**2
    print(f"Final Size of DataFrame after optimizin numeric columns: {final_size:.2f} MB")
    print(f"Memory reduction: {(initial_size - final_size):.2f} MB ({100 * (initial_size - final_size) / initial_size:.2f}%)")
    return df

# Aplicar la optimización numérica al DataFrame correcto
df_combined_filtered_reduced = optimize_numeric_columns_precise(df_combined_filtered_reduced)


Initial Size of the frame: 20933.61 MB
Final Size of DataFrame after optimizin numeric columns: 19101.04 MB
Memory reduction: 1832.58 MB (8.75%)


In [67]:
# Convert the selected columns to categories
columns_to_convert = [
    'exclusion_flag', 'frequency_flag', 'loyalty_flag', 'spender_flag',
    'family_status', 'state', 'Region', 'gender', 'department_id'
]

for col in columns_to_convert:
    df_combined_filtered_reduced[col] = df_combined_filtered_reduced[col].astype('category')

# Check the memory usage after conversion
final_size = df_combined_filtered_reduced.memory_usage(deep=True).sum() / 1024**2
print(f" DF Final Size after convert selected colums to categoria: {final_size:.2f} MB")


 DF Final Size after convert selected colums to categoria: 4816.71 MB


In [65]:
print(df_combined_filtered_reduced.dtypes)


order_number                 int8
order_dow                    int8
order_hour_of_day            int8
department_id            category
prices                    float32
busiest_day                object
busiest_period_of_day      object
max_order                    int8
loyalty_flag             category
average_spend             float32
spender_flag             category
Customer_frequency        float32
frequency_flag           category
gender                   category
state                    category
age                          int8
num_dependents               int8
family_status            category
income                      int32
Region                   category
exclusion_flag           category
dtype: object


In [71]:
# Verifiying Size per column
column_sizes = df_combined_filtered_reduced.memory_usage(deep=True).sort_values(ascending=False) / 1024**2
print("Memory usage per column (in MB):")
print(column_sizes)


Memory usage per column (in MB):
busiest_period_of_day    1832.577055
busiest_day              1831.373729
Index                     236.461555
income                    118.230778
prices                    118.230778
average_spend             118.230778
Customer_frequency        118.230778
state                      29.562506
department_id              29.558385
family_status              29.558104
Region                     29.558069
frequency_flag             29.557989
loyalty_flag               29.557978
spender_flag               29.557913
gender                     29.557900
exclusion_flag             29.557868
order_number               29.557694
age                        29.557694
num_dependents             29.557694
max_order                  29.557694
order_hour_of_day          29.557694
order_dow                  29.557694
dtype: float64


## 1.3 - Action 3 - Changing the type of selected variables

In [94]:
# Verify values of 'busiest_period_of_day' y 'busiest_day'
columns_to_check = ['busiest_period_of_day', 'busiest_day']

for col in columns_to_check:
    print(f"\nCounts per column '{col}':")
    print(df_combined_filtered_reduced[col].value_counts())



Counts per column 'busiest_period_of_day':
busiest_period_of_day
Fewest orders    30993489
Name: count, dtype: int64

Counts per column 'busiest_day':
busiest_day
Regularly busy    21451175
Busiest day        5913988
Least busy         3628326
Name: count, dtype: int64


In [96]:
# Verify current type of columns
print(df_combined_filtered_reduced[['busiest_period_of_day', 'busiest_day']].dtypes)

# Covert both columns to categories
df_combined_filtered_reduced['busiest_period_of_day'] = df_combined_filtered_reduced['busiest_period_of_day'].astype('category')
df_combined_filtered_reduced['busiest_day'] = df_combined_filtered_reduced['busiest_day'].astype('category')

# Verify data typed after convert
print("\nNew Data type after change:")
print(df_combined_filtered_reduced[['busiest_period_of_day', 'busiest_day']].dtypes)


busiest_period_of_day    category
busiest_day              category
dtype: object

New Data type after change:
busiest_period_of_day    category
busiest_day              category
dtype: object


In [98]:
# Verifiying Size per column
column_sizes = df_combined_filtered_reduced.memory_usage(deep=True).sort_values(ascending=False) / 1024**2
print("Memory usage per column (in MB):")
print(column_sizes)

Memory usage per column (in MB):
Index                    236.461555
Customer_frequency       118.230778
income                   118.230778
prices                   118.230778
average_spend            118.230778
state                     29.562506
department_id             29.558385
family_status             29.558104
Region                    29.558069
frequency_flag            29.557989
loyalty_flag              29.557978
busiest_day               29.557971
spender_flag              29.557913
gender                    29.557900
exclusion_flag            29.557868
busiest_period_of_day     29.557857
order_number              29.557694
max_order                 29.557694
age                       29.557694
num_dependents            29.557694
order_hour_of_day         29.557694
order_dow                 29.557694
dtype: float64


In [100]:
print(df_combined_filtered_reduced.dtypes)


order_number                 int8
order_dow                    int8
order_hour_of_day            int8
department_id            category
prices                    float32
busiest_day              category
busiest_period_of_day    category
max_order                    int8
loyalty_flag             category
average_spend             float32
spender_flag             category
Customer_frequency        float32
frequency_flag           category
gender                   category
state                    category
age                          int8
num_dependents               int8
family_status            category
income                      int32
Region                   category
exclusion_flag           category
dtype: object


In [104]:
# Review and convert float columns to integers if possible.
float_columns = ['prices', 'average_spend', 'Customer_frequency']
for col in float_columns:
    # Verificar si la columna puede convertirse a enteros
    if (df_combined_filtered_reduced[col] % 1 == 0).all():  # Si no tiene decimales
        df_combined_filtered_reduced[col] = df_combined_filtered_reduced[col].astype('int32')

# Redcue column 'income'
income_min = df_combined_filtered_reduced['income'].min()
income_max = df_combined_filtered_reduced['income'].max()

if income_min > -32768 and income_max < 32767:
    df_combined_filtered_reduced['income'] = df_combined_filtered_reduced['income'].astype('int16')
elif income_min > -128 and income_max < 127:
    df_combined_filtered_reduced['income'] = df_combined_filtered_reduced['income'].astype('int8')

# Verify the df size
final_size = df_combined_filtered_reduced.memory_usage(deep=True).sum() / 1024**2
print(f"Tamaño total del DataFrame después de optimizar float32 e int32: {final_size:.2f} MB")


Tamaño total del DataFrame después de optimizar float32 e int32: 1211.87 MB


In [106]:
# Verifiying Size per column
column_sizes = df_combined_filtered_reduced.memory_usage(deep=True).sort_values(ascending=False) / 1024**2
print("Memory usage per column (in MB):")
print(column_sizes)

Memory usage per column (in MB):
Index                    236.461555
Customer_frequency       118.230778
income                   118.230778
prices                   118.230778
average_spend            118.230778
state                     29.562506
department_id             29.558385
family_status             29.558104
Region                    29.558069
frequency_flag            29.557989
loyalty_flag              29.557978
busiest_day               29.557971
spender_flag              29.557913
gender                    29.557900
exclusion_flag            29.557868
busiest_period_of_day     29.557857
order_number              29.557694
max_order                 29.557694
age                       29.557694
num_dependents            29.557694
order_hour_of_day         29.557694
order_dow                 29.557694
dtype: float64


In [108]:
print(df_combined_filtered_reduced.dtypes)


order_number                 int8
order_dow                    int8
order_hour_of_day            int8
department_id            category
prices                    float32
busiest_day              category
busiest_period_of_day    category
max_order                    int8
loyalty_flag             category
average_spend             float32
spender_flag             category
Customer_frequency        float32
frequency_flag           category
gender                   category
state                    category
age                          int8
num_dependents               int8
family_status            category
income                      int32
Region                   category
exclusion_flag           category
dtype: object


# Exporting optimized data

In [118]:
file_path = r'C:\Users\cmfag\OneDrive\Desktop\Data Analyst Specialist\Achievement 4\02 DATA\Prepared Data\df_combined_filtered_reduced.pkl'

# Save the DataFrame as a pickle file
df_combined_filtered_reduced.to_pickle(file_path)

# Confirm successful save
print(f"Saved at: {file_path}")


Saved at: C:\Users\cmfag\OneDrive\Desktop\Data Analyst Specialist\Achievement 4\02 DATA\Prepared Data\df_combined_filtered_reduced.pkl
